## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install -c conda-forge gmaps

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)
import matplotlib.pyplot as plt

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,New Norfolk,AU,-42.7826,147.0587,53.74,80,79,0.92,broken clouds
1,1,Vestmannaeyjar,IS,63.4427,-20.2734,54.59,47,0,3.44,clear sky
2,2,Albany,US,42.6001,-73.9662,64.81,82,100,2.42,light rain
3,3,Pidhorodna,UA,48.1070,30.8895,57.99,44,3,7.16,clear sky
4,4,Belmonte,BR,-15.8631,-38.8828,84.20,66,33,13.33,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Belmonte,BR,-15.8631,-38.8828,84.20,66,33,13.33,scattered clouds
6,6,Buala,SB,-8.1450,159.5921,79.11,84,17,3.18,few clouds
7,7,Ribeira Grande,PT,38.5167,-28.7000,75.60,83,40,11.50,scattered clouds
9,9,Kahului,US,20.8947,-156.4700,77.85,83,75,0.00,broken clouds
11,11,Mastic Beach,US,40.7668,-72.8521,76.57,69,1,5.01,clear sky
14,14,Bengkulu,ID,-3.8004,102.2655,77.16,90,92,7.49,overcast clouds
16,16,Sur,OM,22.5667,59.5289,85.57,71,100,18.79,overcast clouds
17,17,Mahebourg,MU,-20.4081,57.7000,73.26,94,75,10.36,light rain
19,19,Padang,ID,-0.9492,100.3543,78.85,87,97,3.60,overcast clouds
20,20,Naze,JP,28.3667,129.4833,77.99,61,0,17.60,clear sky


In [7]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Belmonte,BR,-15.8631,-38.8828,84.20,66,33,13.33,scattered clouds
6,6,Buala,SB,-8.1450,159.5921,79.11,84,17,3.18,few clouds
7,7,Ribeira Grande,PT,38.5167,-28.7000,75.60,83,40,11.50,scattered clouds
9,9,Kahului,US,20.8947,-156.4700,77.85,83,75,0.00,broken clouds
11,11,Mastic Beach,US,40.7668,-72.8521,76.57,69,1,5.01,clear sky
...,...,...,...,...,...,...,...,...,...,...
671,671,Villa Del Rosario,CO,7.8339,-72.4742,88.18,51,20,8.05,few clouds
673,673,San Antonio,US,29.4241,-98.4936,83.10,74,1,3.44,clear sky
680,680,Hamilton,US,39.1834,-84.5333,72.00,93,90,1.99,mist
682,682,Sibut,CF,5.7180,19.0739,76.24,89,86,3.96,light rain


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Belmonte,BR,84.20,scattered clouds,-15.8631,-38.8828,
6,Buala,SB,79.11,few clouds,-8.1450,159.5921,
7,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,
9,Kahului,US,77.85,broken clouds,20.8947,-156.4700,
11,Mastic Beach,US,76.57,clear sky,40.7668,-72.8521,
14,Bengkulu,ID,77.16,overcast clouds,-3.8004,102.2655,
16,Sur,OM,85.57,overcast clouds,22.5667,59.5289,
17,Mahebourg,MU,73.26,light rain,-20.4081,57.7000,
19,Padang,ID,78.85,overcast clouds,-0.9492,100.3543,
20,Naze,JP,77.99,clear sky,28.3667,129.4833,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Belmonte,BR,84.20,scattered clouds,-15.8631,-38.8828,Pousada do Diogo
6,Buala,SB,79.11,few clouds,-8.1450,159.5921,Maringe Lagoon Lodge
7,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,Casa da Japoneira
9,Kahului,US,77.85,broken clouds,20.8947,-156.4700,Banana Bungalow Maui Hostel
11,Mastic Beach,US,76.57,clear sky,40.7668,-72.8521,Smith Point County Park Campground


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))